In [14]:
# test_xi_init_real.py
import numpy as np
from collections import Counter

from env import FPLEnv
from data_utils import load_season_fn, load_gw_fn
from xp import load_models

# ---- Adjust these ----
BASE_DIR = "../data/Fantasy-Premier-League"   # path to the vaastav repo clone root
SEASONS  = ["2023-24"]                        # start with one season you have features for
START_GW = 1                                  # GW to start the episode
models = load_models("../models/rand_forest/classifiers")


env = FPLEnv(
    load_season_fn=load_season_fn,  # now returns season_ctx incl. predictor (if you added it)
    load_gw_fn=load_gw_fn,          # forwards predictor to the pool builder
    seasons=["2023-24"],
    base_dir="../data/Fantasy-Premier-League",
    start_gw=2,                     # if you want transfers to start at GW2
    budget=100.0,
    temperature=0.8,
    transfer_hit=-4.0,
    max_free_transfers=5,
    models=models,                  # <-- pass the 4 models here
)
obs, info = env.reset()


In [15]:
env.squad.players

[Player(pid=597, name='André Onana', pos='GK', team='Man Utd', team_id=14, price=5.0, features=(5.017559994, 2.676031997, 0.718888147, 3.310813683, 1.0), pooling_metric=158228.0, xP=0.6438328018980136),
 Player(pid=17, name='Aaron Ramsdale', pos='GK', team='Arsenal', team_id=1, price=5.0, features=(1.115013332, 0.724758666, 0.331034483, 5.0, 1.0), pooling_metric=45504.0, xP=0.7386058918410471),
 Player(pid=430, name='Kieran Trippier', pos='DEF', team='Newcastle', team_id=15, price=6.5, features=(1.115013332, 4.181299995, 0.310206408, 3.310813683, 1.0), pooling_metric=78901.0, xP=0.43464318860064455),
 Player(pid=313, name='Virgil van Dijk', pos='DEF', team='Liverpool', team_id=11, price=6.0, features=(0.0, 0.0, 0.0, 4.310813683, 1.0), pooling_metric=732873.0, xP=0.6235140979339946),
 Player(pid=131, name='Pervis Estupiñán', pos='DEF', team='Brighton', team_id=5, price=5.1, features=(3.902546662, 8.920106656, 0.910709341, 2.506614033, 1.0), pooling_metric=361772.0, xP=0.6534819320138483

In [16]:
import numpy as np

players = env.squad.players

# current best XI from reset/info
xi_idx = info["xi_indices"]

# choose captain & vice from current XI (top-2 xP)
xi_sorted = sorted(xi_idx, key=lambda i: players[i].xP, reverse=True)
cap_idx = xi_sorted[0]
vc_idx  = xi_sorted[1] if len(xi_sorted) > 1 else xi_sorted[0]

# pick your worst XI player by xP to transfer out
out_idx = min(xi_idx, key=lambda i: players[i].xP)
out_p = players[out_idx]

# find best legal incoming from the pool for the same position
pool_pos_idxs = env.index_map[out_p.pos]
in_idx = None
for j in sorted(pool_pos_idxs, key=lambda k: (env.pool[k].xP, -env.pool[k].price), reverse=True):
    if env.squad.can_swap(out_idx, env.pool[j]):
        in_idx = j
        break

# build the action: [out, in, captain, vice]
if in_idx is None:
    print("No legal incoming found; skipping but still choose bands.")
    action = np.array([15, len(env.pool), cap_idx, vc_idx], dtype=int)  # skip action still needs cap/vc
else:
    print(f"OUT: {out_p.name} (£{out_p.price})  ->  IN: {env.pool[in_idx].name} (£{env.pool[in_idx].price})")
    action = np.array([out_idx, in_idx, cap_idx, vc_idx], dtype=int)

obs, reward, done, truncated, info = env.step(action)

print("\nAfter step:")
print(" GW:", info["current_gw"])
print(" Reward:", round(reward, 3))
print(" Bank: £{:.1f}m | FTs: {}".format(info["bank"], info["free_transfers"]))
print(" Captain / Vice:", info["captain"], "/", info["vice_captain"])
print(" XI size:", len(info["xi_indices"]))


OUT: Kieran Trippier (£6.5)  ->  IN: Kyle Walker (£5.0)

After step:
 GW: 3
 Reward: 6.0
 Bank: £1.5m | FTs: 0
 Captain / Vice: Mohamed Salah / Erling Haaland
 XI size: 11


In [17]:
print(env.current_gw)
env.squad.players

3


[Player(pid=597, name='André Onana', pos='GK', team='Man Utd', team_id=14, price=5.0, features=(3.772517915, 3.222299802, 0.471577694, 3.689186317, 1.0), pooling_metric=117188.0, xP=0.633804371865569),
 Player(pid=17, name='Aaron Ramsdale', pos='GK', team='Arsenal', team_id=1, price=5.0, features=(3.130553692, 1.044622425, 0.410793002, 5.0, 1.0), pooling_metric=53491.0, xP=0.6421316529747475),
 Player(pid=369, name='Kyle Walker', pos='DEF', team='Man City', team_id=13, price=5.1, features=(4.440473701, 1.810237059, 0.906307778, 5.0, 1.0), pooling_metric=315193.0, xP=0.7403375072072906),
 Player(pid=313, name='Virgil van Dijk', pos='DEF', team='Liverpool', team_id=11, price=6.0, features=(0.0, 0.0, 0.0, 4.310813683, 1.0), pooling_metric=732873.0, xP=0.6235140979339946),
 Player(pid=131, name='Pervis Estupiñán', pos='DEF', team='Brighton', team_id=5, price=5.2, features=(6.830948443, 9.695748672, 0.967666024, 2.804199649, 1.0), pooling_metric=806778.0, xP=0.667046537084024),
 Player(pid=